In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
import pandas as pd
import scanpy as sc
import anndata as ad
from sklearn.cluster import AgglomerativeClustering, KMeans
from sklearn.metrics import adjusted_rand_score
import community
import networkx as nx
import gzip
from scipy.sparse import csc_matrix, csr_matrix
%matplotlib inline

from utils.io import create_AnnData_from_gem
from utils.routines import delete_module
import utils.io as io

from sklearn.cluster import KMeans

In [2]:
andata_binned_50_C5 = io.create_binned_anndata_from_gem('/goofys/Samples/batch_effects_removal_dataset/raw/mouse_embryo_bin1/SS200001209TR_C5.tissue.gem.gz', 50)
andata_binned_50_C6 = io.create_binned_anndata_from_gem('/goofys/Samples/batch_effects_removal_dataset/raw/mouse_embryo_bin1/SS200001209TR_C6.tissue.gem.gz', 50)
andata_binned_50_E4 = io.create_binned_anndata_from_gem('/goofys/Samples/batch_effects_removal_dataset/raw/mouse_embryo_bin1/SS200000842BL_E4.tissue.gem.gz', 50)

In [ ]:
adatas = [andata_binned_50_C5, andata_binned_50_C6, andata_binned_50_E4]

In [ ]:
def merde_datasets(adatas): #list of anndatas
    genes = [adata.var_names.values for adata in adatas]
    datasets =  [adata.X for adata in adatas]

    keep_genes = set()
    for idx, gene_list in enumerate(genes):
        keep_genes |= set(gene_list)

    
    union_genes = sorted(keep_genes)
    for i in range(len(datasets)):
        X_new = np.zeros((datasets[i].shape[0], len(union_genes)))
        X_old = csc_matrix(datasets[i])
        gene_to_idx = { gene: idx for idx, gene in enumerate(genes[i]) }
        for j, gene in enumerate(union_genes):
            if gene in gene_to_idx:
                X_new[:, j] = X_old[:, gene_to_idx[gene]].toarray().flatten()
        datasets[i] = csr_matrix(X_new) #to mi treba

    ret_genes = np.array(union_genes)

    new_adatas = []
    for i in range(len((adatas))): #3
        adata = ad.AnnData(datasets[i])
        adata.obs = adatas[i].obs #ovo ce mi trebati da bih imala koordinate
        
        new_adatas.append(adata)

    andata_merged = sc.concat(new_adatas,label="sample_label") #ovo automatski dodeljuje labelu batchu

    return andata_merged

In [ ]:
def merging_and_clustering(adata_list): #merging, preprocessing, clustering and visualization
    
    #merging
    andata_merged=merde_datasets(adata_list)
    andata_merged_copy = andata_merged.copy()
    
    #Normalize
    sc.pp.normalize_total(andata_merged_copy, target_sum=1e4)
    sc.pp.log1p(andata_merged_copy)
    sc.pp.highly_variable_genes(andata_merged_copy,n_top_genes=andata_merged_copy.shape[1],subset=True) #selektuje sve gene (kao da nemam to uopste)
    sc.pp.scale(andata_merged_copy, max_value=10)
    
    #dim reduction
    sc.tl.pca(andata_merged_copy,n_comps=100)

    #clustering - leiden
    sc.pp.neighbors(andata_merged_copy,random_state=0, use_rep="X_pca")
    sc.tl.leiden(andata_merged_copy, resolution=1, key_added="leiden")

    sc.tl.umap(andata_merged_copy) #visualization
    sc.pl.umap(andata_merged_copy, color=['leiden'], size=5)
    sc.tl.dendrogram(andata_merged_copy, groupby = "leiden")
    sc.pl.dendrogram(andata_merged_copy, groupby = "leiden")

    #kmeans
    #extract pca coordinates
    X_pca = andata_merged_copy.obsm['X_pca'] 

    # kmeans with k=38
    kmeans = KMeans(n_clusters=38, random_state=0).fit(X_pca) 
    andata_merged_copy.obs['kmeans'] = kmeans.labels_.astype(str)
    
    sc.pl.umap(andata_merged_copy, color=['kmeans'], size=5)
    sc.tl.dendrogram(andata_merged_copy, groupby = "kmeans")
    sc.pl.dendrogram(andata_merged_copy, groupby = "kmeans")

    #scatial coordinates
    for i in range(len(adata_list)):
        dataset = andata_merged_copy.obs[andata_merged_copy.obs['sample_label']==str(i)]
        plt.figure()
        plt.scatter(dataset['y'].values,-dataset['x'].values, c=dataset['leiden'].values.astype('int64'), cmap='jet', s=2)
        plt.title('Leiden clustering of sample' + str(i))
        plt.figure()
        plt.scatter(dataset['y'].values,-dataset['x'].values, c=dataset['kmeans'].values.astype('int64'), cmap='jet', s=2)
        plt.title('Kmeans clustering of sample' + str(i))


    return andata_merged_copy

In [ ]:
adata = merging_and_clustering(adatas)